In [ ]:
# Import library

# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
from deconvbench import Deconvbench
from mpl_toolkits.axes_grid1 import make_axes_locatable
from amiral import instructment, utils, parameter, gradient, minimisation, array, plotting


# from plotting import plot_PSF_PSD as amiral_plt
from scipy.optimize import minimize 

%matplotlib inline

import tools

rcParams["figure.figsize"] = 20,33

In [ ]:
# Function for getting snr
def get_snr (array, noise):
    
    mean = np.mean(array)
    sig2 = np.std(noise)
    
    snr = mean / sig2
    
    return snr

def psd_object (param):
    
    rho = np.fft.fftshift(utils.dist(768))/param[1]
    psd_obj =  param[0]/ (np.power(rho,param[2]) + 1.)
    
    return psd_obj
    
def plot_psd_object(psd_obj): 
    
    fig, ax = plt.subplots()
    ycent = int((256*aosys_cls.samp_factor[0])//2)

    ax.plot(np.abs(psd_obj[ycent,...]))
    ax.set_title('PSD Object(total)')
    ax.axhline(y=1, color = 'r', ls = '--')
    
    pass

def create_psfao19_otf (otf_tel, guess, aosys_cls): 
    
    # Use PSFAO19 model to create a PSF
    psd_ao = aosys_cls.psd_residual_ao (guess = guess)
    psd_halo = aosys_cls.psd_residual_halo(r0 = guess[0])
    
    psd = psd_ao + psd_halo

    otf_atmo = aosys_cls.otf_atmo(psd)
    otf_total = otf_atmo*otf_tel
      
    return otf_atmo,otf_total

In [ ]:
# PATH

wdir = "/Users/alau/Data/amiral_fits/VESTA/SNR/"
path_asteriod = '/Users/alau/IDLWorkspace/Data/Vesta_OASIS/2018-06-08T07_58_51.472.fits'

# Parameters for the PSF
psf_dict = {
    "r0": 0.15,                  
    "background": 0.,      
    "amplitude": 0.5,       
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 0. 
}

# Variable
RON = 10. # CCD read-out noise standard-deviation [e-]
FLUX = np.linspace(5e6,5e9,10)
DIMENSION = int(256)

flux_snr = FLUX[9]
print(flux_snr)

# psf_dict["r0"] = 0.1
# psf_dict["amplitude"] = 0.5

Get the object from the high resolution simulation

In [ ]:
data = fits.open(path_asteriod)
asteriod = data[0].data

asteriod_resize = np.zeros((DIMENSION,DIMENSION))


cuta = DIMENSION//2-128
cutb = DIMENSION//2+128
 
asteriod_resize[cuta:cutb,cuta:cutb] = asteriod
asteriod_resize = asteriod_resize/np.sum(asteriod_resize)

# Calibrating the flux
asteriod_resize = asteriod_resize/np.sum(asteriod_resize)*flux_snr
plt.imshow(asteriod_resize)

Get some more objects

In [ ]:
asteriod_resize_1 = np.zeros((DIMENSION*2,DIMENSION*2))


cuta = DIMENSION*2//2-128
cutb = DIMENSION*2//2+128


asteriod_resize_1[cuta:cutb,cuta:cutb] = asteriod
asteriod_resize_1 = asteriod_resize_1/np.sum(asteriod_resize_1)

# Calibrating the flux
asteriod_resize_1 = asteriod_resize_1/np.sum(asteriod_resize_1)*flux_snr

plt.imshow(asteriod_resize_1)

In [ ]:
asteriod_resize_2 = np.zeros((DIMENSION*3,DIMENSION*3))


cuta = DIMENSION*3//2-128
cutb = DIMENSION*3//2+128


asteriod_resize_2[cuta:cutb,cuta:cutb] = asteriod
asteriod_resize_2 = asteriod_resize_2/np.sum(asteriod_resize_2)

# Calibrating the flux
asteriod_resize_2 = asteriod_resize_2/np.sum(asteriod_resize_2)*flux_snr

plt.imshow(asteriod_resize_2)

We need a PSF

In [ ]:
# Set up the telescope and produce a PSF
amiral_dict = {
    "r0": 0.2,                  
    "background": 0.,      
    "amplitude": 0.5,       
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 0. 
}

aosys_dict = {
    'diameter': 8 , 
    'occ_ratio': 0.14 , 
    'no_acutuator' : 20, 
    'wavelength': 500, 
    'dimension': 256,
    'resolution_rad' : 2.083e-8
}

# Passing parametpsd_arrayers from the telesope to the aosystem
aosys_cls = instructment.aoSystem( 
        diameter = aosys_dict['diameter'], occ_ratio = aosys_dict['occ_ratio'], 
        no_acutuator= aosys_dict['no_acutuator'], wavelength = aosys_dict['wavelength']*1e-9, 
        resolution_rad = aosys_dict['resolution_rad'], 
        dimension=aosys_dict['dimension'])  


psf_keys, psf_param = utils.dict2array(psf_dict)
amiral_keys, psf_guess = utils.dict2array(amiral_dict)

In [ ]:
psd_ao = aosys_cls.psd_residual_ao (psf_param)
psd_halo = aosys_cls.psd_residual_halo(psf_dict['r0'])

psd = psd_halo + psd_ao 

pupil = aosys_cls.get_pupil_plane()
otf_tel = aosys_cls.pupil_to_otf_tel(pupil)

integral, SR = aosys_cls.psd_integral(psd_ao, psf_dict['r0'])

otf_atmo = aosys_cls.otf_atmo(psd)

otf_total = np.fft.ifftshift(otf_atmo * otf_tel)

psf_total = np.fft.ifft2(otf_total)

psf_tel = np.fft.ifft2(otf_tel)

plt.imshow(np.real(np.log10(otf_total)))

Get Gaussian Noise

In [ ]:
# Noise 
gauss_noise = np.random.randn(np.shape(asteriod_resize)[0],np.shape(asteriod_resize)[0])*RON

rng = np.random.default_rng()
photon_noise = rng.poisson(gal_resize)

noise = gauss_noise + photon_noise

In [ ]:
ft_obj = np.fft.fft2(asteriod_resize)
ft_image = ft_obj* otf_total

plt.imshow(np.log10(np.real(ft_image)))

Add Photon noise

In [ ]:
# rng = np.random.default_rng()
# photon_noise = rng.poisson(asteriod_resize)
# print("\nPhoton Noise of the object: ", np.sum((photon_noise)))

# ft_photon_noise = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(photon_noise)))
ft_obj = np.fft.fft2(np.fft.ifftshift(asteriod_resize))